# Отток клиентов

### Введение

Из банка стали уходить клиенты. Каждый месяц. Немного, но заметно. Маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.
Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Нам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Построим модель с предельно большим значением *F1*-меры. Доведем метрику до 0.59.  
Дополнительно измерим *AUC-ROC*, сравнивая её значение с *F1*-мерой.

Описание данных  

Признаки  
•	RowNumber — индекс строки в данных  
•	CustomerId — уникальный идентификатор клиента  
•	Surname — фамилия  
•	CreditScore — кредитный рейтинг  
•	Geography — страна проживания  
•	Gender — пол  
•	Age — возраст  
•	Tenure — количество недвижимости у клиента  
•	Balance — баланс на счёте  
•	NumOfProducts — количество продуктов банка, используемых клиентом  
•	HasCrCard — наличие кредитной карты  
•	IsActiveMember — активность клиента  
•	EstimatedSalary — предполагаемая зарплата  
Целевой признак  
•	Exited — факт ухода клиента  

План работы:
1. Подготовка данных.  
2. Исследование баланса классов, обучение модели без учёта дисбаланса.  
3. Улучшение качества модели с учетом дисбаланса классов. Обучение разных моделей, выбор лучшей.  
4. Финальное тестирование.
5. Общий вывод.



### 1. Подготовка данных

### 1.1. Предобработка данных

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('C:\\dm\\DM documents\\Data Science\\Курс 5 - ОБУЧЕНИЕ С УЧИТЕЛЕМ\\Проект ОТТОК КЛИЕНТОВ ИЗ БАНКА\\Churn.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
# проверка на наличие пропусков, проверка типа данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [5]:
# поиск дупликатов строк
df.duplicated().sum()

0

In [6]:
# проверка на наличие смысловых дупликатов в столбце 'Geography'
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [7]:
# проверка на наличие смысловых дупликатов в столбце 'Gender'
df['Gender'].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

In [8]:
# заполним пропуски 'Tenure' (количество объектов недвижимости) медианным значением в зависимости от возраста
def tenure_guess(row):
    med = df[(df['Age']==row['Age'])&(df['Tenure'].isna()==False)]['Tenure'].median()
    return med
df.loc[df['Tenure'].isna(), 'Tenure']= df[df['Tenure'].isna()].apply(tenure_guess, axis=1)

In [9]:
# удалим столбцы, ненужные в данной работе
df = df.drop(columns=['RowNumber','CustomerId','Surname'])

#### Вывод по предобработке данных   

Были обнаружены пропуски в данных о количестве объектов недвижимости у клиента. Т.к. количество пропусков значительное - 9% - , то пропуски были заполнены. Для заполнения пропусков использовали медианное значение количества объектов недвижимости клиентов соответствующего возраста.  
Дупликатов не обнаружено. Изменение типа данных не требуется.  
Из таблицы удалили столбцы, не имеющие отношения к данной работе.

### 1.2. Подготовка признаков.

#### Кодирование - преобразование категориальных признаков в численные.

Воспользуемся методом прямого кодирования, как более универсальным - есть возможность применить этот метод к логистической регрессии.

In [10]:
# замена столбцов 'Geography' и 'Gender' на фиктивные переменные (применили параметр drop_first, чтобы избежать dummy-ловушки)
dfo = pd.get_dummies(df, drop_first=True, columns=['Geography', 'Gender'])
dfo.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


#### Масштабирование признаков

Значения и их разбросы в разных столбцах различаются значительно. Поэтому необходимо произвести масштабирование.

Масштабирование проведем после разделения данных на три выборки. Сначала настроим scaler по обучающей выборке методом fit, а затем уже применим transform ко всем трем выборкам.  
    Дело в том, что скармливая методу fit _всю_ выборку, мы допустим некоторую утечку тестовых данных в обучающие, а за ними и в модель. Параметры тестовых данных, такие как дисперсия, среднее и т.д влияют на обучающую выборку. Таким образом, для чистоты экспериманта модель не должна видеть тестовые данные ни в каком, даже столь косвенном виде.

In [11]:
# импорт функции train_test_split библиотеки sklearn
from sklearn.model_selection import train_test_split

In [12]:
# выделение тестовой выборки
dfo_1, dfo_test = train_test_split(dfo, test_size=0.2, random_state=1)

In [13]:
# выделение обучающей и валидационной выборок
dfo_train, dfo_valid = train_test_split(dfo_1, test_size=0.25, random_state=12345)

In [14]:
print('Длина выборки')
print('- обучающая:   ', len(dfo_train))
print('- валидационная:   ', len(dfo_valid))
print('- тестовая:   ', len(dfo_test))

Длина выборки
- обучающая:    6000
- валидационная:    2000
- тестовая:    2000


In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
scaler = StandardScaler()

In [17]:
# список признаков, которые будем масштабировать
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [18]:
scaler.fit(dfo_train[numeric]) # настроиваем scaler по обучающей выборке
dfo[numeric] = scaler.transform(dfo[numeric]) # применяем transform ко всем трем выборкам

In [19]:
# таблица готова для дальнейшей работы
dfo.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,-0.328134,0.296364,-1.078051,-1.222409,-0.917230,1,1,0.033162,1,0,0,0
1,-0.442328,0.201385,-1.440136,0.124262,-0.917230,0,1,0.228021,0,0,1,0
2,-1.542751,0.296364,1.094465,1.343108,2.507401,1,0,0.252201,1,0,0,0
3,0.502374,0.011429,-1.440136,-1.222409,0.795085,0,0,-0.097785,0,0,0,0
4,2.069957,0.391342,-1.078051,0.794368,-0.917230,1,1,-0.354422,0,0,1,0


In [20]:
dfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
CreditScore          10000 non-null float64
Age                  10000 non-null float64
Tenure               10000 non-null float64
Balance              10000 non-null float64
NumOfProducts        10000 non-null float64
HasCrCard            10000 non-null int64
IsActiveMember       10000 non-null int64
EstimatedSalary      10000 non-null float64
Exited               10000 non-null int64
Geography_Germany    10000 non-null uint8
Geography_Spain      10000 non-null uint8
Gender_Male          10000 non-null uint8
dtypes: float64(6), int64(3), uint8(3)
memory usage: 732.5 KB


#### Вывод по подготовке признаков

Преобразовали категориальные признаки в численные и произвели масштабирование признаков. Применили прямое кодирование (а не порядковое), потому что в исследовании будет участвовать и модель логистической регрессии.

### 2. Исследование баланса классов, обучение модели без учёта дисбаланса.

In [21]:
# баланс классов
class1 = dfo['Exited'].sum()
class0 = dfo[df['Exited']==0]['Exited'].count()
classT = class1+class0
print('Ушли:', class1 )
print('Остались:', class0)
print('Всего:', classT)
print('')
print('Дисбаланс: класс 0 встречается в', round(class0/class1,1),'раза чаще, чем класс 1.')

Ушли: 2037
Остались: 7963
Всего: 10000

Дисбаланс: класс 0 встречается в 3.9 раза чаще, чем класс 1.


In [22]:
# переменные для признаков и целевого признака
features_train = dfo_train.drop(['Exited'], axis=1)
target_train = dfo_train['Exited']
features_valid = dfo_valid.drop(['Exited'], axis=1)
target_valid = dfo_valid['Exited']

In [23]:
# заготовка параметров таблицы результатов исследования моделей
columns = ['model', 'n_estimators', 'depth', 'random_state', 'accuracy', 'f1_score', 'roc_auc_score']
data=[]

In [24]:
# импорт необходимых функций
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [25]:
# модель "решающее дерево" (oграничим глубину дерева max_depth=30)
for md_t in range (6,31,2):
    # создание и обучение модели
    model_t = DecisionTreeClassifier(random_state=12345, max_depth=md_t)
    model_t.fit(features_train, target_train)
    # предсказания для валидационных данных и расчет значения метрик
    predicted_valid_t = model_t.predict(features_valid)
    prob_valid_t = model_t.predict_proba(features_valid)
    prob1_valid_t = prob_valid_t[:,1]
    acc_t = accuracy_score(target_valid, predicted_valid_t)
    f1_score_t = f1_score(target_valid, predicted_valid_t)
    roc_auc_score_t = roc_auc_score(target_valid, prob1_valid_t)
    # добавление строк в таблицу сравнения моделей
    line_tree=['decision_tree',np.nan, md_t, 12345, acc_t, f1_score_t, roc_auc_score_t]
    data.append(line_tree)

In [26]:
%%time

# модель "случайный лес" 
# переберем значения n_estimators,
# oграничим макс глубину деревьев max_depth,
for md_f in range (5, 31, 5):
    for estim in range (10, 101, 20):
        # создание и обучение модели
        model_f = RandomForestClassifier (n_estimators=estim, max_depth=md_f, random_state=12345)
        model_f.fit(features_train, target_train)
        # предсказания для валидационных данных и расчет значения метрик
        predicted_valid_f = model_f.predict(features_valid)
        prob_valid_f = model_f.predict_proba(features_valid)
        prob1_valid_f = prob_valid_f[:,1]
        acc_f = accuracy_score(target_valid, predicted_valid_f)
        f1_score_f = f1_score(target_valid, predicted_valid_f)
        roc_auc_score_f = roc_auc_score(target_valid, prob1_valid_f)
        # добавление строк в таблицу сравнения моделей
        line_forest = ['random_forest', estim, md_f, 12345, acc_f, f1_score_f, roc_auc_score_f]
        data.append(line_forest)

Wall time: 23.8 s


In [27]:
# модель "логистическая регрессия"
# создание и обучение модели
model_lr = LogisticRegression(random_state=12345, solver='lbfgs')
model_lr.fit(features_train, target_train)
# предсказания для валидационных данных и расчет значения метрик
predicted_valid_lr = model_lr.predict(features_valid)
prob_valid_lr = model_lr.predict_proba(features_valid)
prob1_valid_lr = prob_valid_lr[:,1]
acc_lr = accuracy_score(target_valid, predicted_valid_lr)
f1_score_lr = f1_score(target_valid, predicted_valid_lr)
roc_auc_score_lr = roc_auc_score(target_valid, prob1_valid_lr)
# добавление строки в таблицу сравнения моделей
line_lr=['logistic_regression',np.nan, np.nan, 12345, acc_lr, f1_score_lr, roc_auc_score_lr]
data.append(line_lr)

In [28]:
# вывод на экран таблицы сравнения моделей
summary_ib = pd.DataFrame(data=data, columns=columns)
summary_ib

,model,n_estimators,depth,random_state,accuracy,f1_score,roc_auc_score
0,decision_tree,NaN,6.0,12345,0.8380,0.520710,0.831310
1,decision_tree,NaN,8.0,12345,0.8425,0.566713,0.777386
2,decision_tree,NaN,10.0,12345,0.8245,0.539974,0.749402
3,decision_tree,NaN,12.0,12345,0.8050,0.507576,0.683618
4,decision_tree,NaN,14.0,12345,0.7940,0.501211,0.681145
5,decision_tree,NaN,16.0,12345,0.7850,0.492925,0.674543
6,decision_tree,NaN,18.0,12345,0.7850,0.495305,0.682176
7,decision_tree,NaN,20.0,12345,0.7740,0.478060,0.669061
8,decision_tree,NaN,22.0,12345,0.7735,0.472643,0.665783
9,decision_tree,NaN,24.0,12345,0.7735,0.472643,0.665783


In [29]:
# коэффициент корреляции между roc_auc_score и accuracy и roc_auc_score и f1_score
print('Корреляция между roc_auc_score и accuracy:', round(summary_ib['roc_auc_score'].corr(summary_ib['accuracy']),2))
print('Корреляция между roc_auc_score и f1_score:', round(summary_ib['roc_auc_score'].corr(summary_ib['f1_score']),2))

Корреляция между roc_auc_score и accuracy: 0.98
Корреляция между roc_auc_score и f1_score: 0.6


In [30]:
# выбор лучшей модели
best_ib=summary_ib[summary_ib['f1_score']==summary_ib['f1_score'].max()].reset_index(drop=True).head(1)

#### Вывод по исследованию моделей без учета дисбаланса классов

Лучшей оказалась модель со следующими параметрами и характеристиками:

In [31]:
best_ib

,model,n_estimators,depth,random_state,accuracy,f1_score,roc_auc_score
0,random_forest,30.0,25.0,12345,0.858,0.586006,0.830329


Сохраним значения параметров и метрик лучшей модели в отдельных переменных. Они пригодятся в
дальнейшей работе.

In [32]:
estim_best_ib = best_ib.loc[0,'n_estimators'].astype('int64')
md_best_ib = best_ib.loc[0,'depth'].astype('int64')
acc_best_ib = best_ib.loc[0,'accuracy']
f1_score_best_ib = best_ib.loc[0,'f1_score']
roc_auc_score_best_ib = best_ib.loc[0,'roc_auc_score']

### 3. Улучшение качества модели с учетом дисбаланса классов. Обучение разных моделей, выбор лучшей.

Попробуем улучшить качество модели тремя разными способами - взвешивание классов, уменьшение выборки, увеличение выборки.   
В каждом случае произведем подбор гиперпараметров.

In [33]:
# заготовка параметров таблицы результатов применения различных способов борьбы с дисбалансом
columns1 = ['model', 'n_estimators', 'depth', 'method', 'coefficient','accuracy', 'f1_score', 'roc_auc_score']
data1 =[['random_forest', estim_best_ib, md_best_ib, 'imbalanced',np.nan, acc_best_ib, f1_score_best_ib, roc_auc_score_best_ib]]

#### Взвешивание классов

In [34]:
for md in range(5, 31, 5):
    for estim in range(10, 101, 20):
        # создание и обучение модели
        model_cw = RandomForestClassifier(n_estimators=estim, max_depth=md, random_state=12345, class_weight = 'balanced')
        model_cw.fit(features_train, target_train)
        # предсказания для валидационных данных и значения метрик
        predicted_valid_cw = model_cw.predict(features_valid)
        prob_valid_cw = model_cw.predict_proba(features_valid)
        prob1_valid_cw = prob_valid_cw[:,1]
        acc_valid_cw = accuracy_score(target_valid, predicted_valid_cw)
        f1_score_valid_cw = f1_score(target_valid, predicted_valid_cw)
        roc_auc_score_valid_cw = roc_auc_score(target_valid, prob1_valid_cw)
        # добавление соответсвующих строк в таблицу сравнения моделей                              
        line_cw = ['random_forest', estim, md, 'class_weight',np.nan, acc_valid_cw, f1_score_valid_cw, roc_auc_score_valid_cw]
        data1.append(line_cw)

#### Увеличение и уменьшение выборки

In [35]:
# импорт фунции перемешивания
from sklearn.utils import shuffle

In [36]:
# разделение обучающей выборки на отрицательные и положительные объекты
dfo_train_zeros = dfo_train[dfo_train['Exited']==0]
dfo_train_ones = dfo_train[dfo_train['Exited']==1]
features_zeros = dfo_train_zeros.drop(columns=['Exited'])
target_zeros = dfo_train_zeros['Exited']
features_ones = dfo_train_ones.drop(columns=['Exited'])
target_ones = dfo_train_ones['Exited']

In [37]:
# введем вспомогательный целочисленный коэффициент r, 
# показывающий во сколько раз целевых признаков класса "0" больше, чем признаков класса "1" 
r = int(class0/class1)
r

3

In [38]:
for md in range (5, 31, 5):
    for estim in range (10, 101, 20):
        # создание модели
        model_s = RandomForestClassifier(n_estimators=estim, max_depth=md, random_state=12345)
        for met in ['upsampling','downsampling']:
            # исследуем модель при изменении выборки класса "1" в различное число раз
            for k in range (r, r*r):
                if met == 'upsampling':
                    # копирование положительных объектов, создание новой обучающей выборки
                    features_sampled = pd.concat([features_zeros]+[features_ones]*k)
                    target_sampled = pd.concat([target_zeros]+[target_ones]*k)
                    coef = k
                if met == 'downsampling':
                    # коэффициент уменьшения выборки
                    f=k/10
                    # создание новой обучающей выборки
                    features_sampled = pd.concat([features_zeros.sample(frac=f, random_state=12345)] + [features_ones])
                    target_sampled = pd.concat([target_zeros.sample(frac=f, random_state=12345)] + [target_ones])
                    coef = f
                # перемешивание новой обучающей выборки
                features_sampled, target_sampled = shuffle(features_sampled, target_sampled, random_state=12345)
                # обучение модели
                model_s.fit(features_sampled, target_sampled)
                # предсказания для валидационных данных и значения метрик
                predicted_valid_s = model_s.predict(features_valid)
                prob_valid_s = model_s.predict_proba(features_valid)
                prob1_valid_s = prob_valid_s[:,1]
                acc_valid_s = accuracy_score(target_valid, predicted_valid_s)
                f1_score_valid_s = f1_score(target_valid, predicted_valid_s)
                roc_auc_score_valid_s = roc_auc_score(target_valid, prob1_valid_s)
                # добавление соответсвующих строк в таблицу сравнения моделей
                line_s = ['random_forest', estim, md, met, coef, acc_valid_s, f1_score_valid_s, roc_auc_score_valid_s]
                data1.append(line_s)

In [39]:
# таблицa сравнения моделей
summary_b = pd.DataFrame(data=data1, columns=columns1)
summary_b

,model,n_estimators,depth,method,coefficient,accuracy,f1_score,roc_auc_score
0,random_forest,30,25,imbalanced,NaN,0.8580,0.586006,0.830329
1,random_forest,10,5,class_weight,NaN,0.7875,0.581281,0.827768
2,random_forest,30,5,class_weight,NaN,0.7895,0.590068,0.832421
3,random_forest,50,5,class_weight,NaN,0.7895,0.590068,0.837635
4,random_forest,70,5,class_weight,NaN,0.7895,0.591659,0.839568
...,...,...,...,...,...,...,...,...
386,random_forest,90,30,downsampling,0.4,0.8200,0.589977,0.832900
387,random_forest,90,30,downsampling,0.5,0.8330,0.591687,0.837266
388,random_forest,90,30,downsampling,0.6,0.8405,0.591549,0.833352
389,random_forest,90,30,downsampling,0.7,0.8485,0.593289,0.838348


In [40]:
# выбор лучшей модели
best_b=summary_b[summary_b['f1_score']==summary_b['f1_score'].max()].reset_index(drop=True)

#### Вывод по исследованию моделей с учетом дисбаланса классов

Лучшей оказалась модель "случайный лес" со следующими параметрами и характеристиками:

In [41]:
best_b

,model,n_estimators,depth,method,coefficient,accuracy,f1_score,roc_auc_score
0,random_forest,90,30,upsampling,7.0,0.8515,0.619718,0.836449


Сохраним значение коэффициента лучшей модели в отдельной переменной - пригодится в дальнейшей работе.

In [42]:
k_best = best_b.loc[0,'coefficient'].astype('int64')
estim_best = best_b.loc[0,'n_estimators'].astype('int64')
md_best = best_b.loc[0,'depth'].astype('int64')

### 4. Финальное тестирование модели

Обучим выбранную модель на датасете, состоящем из обучающей и валидационной выборок (т.е. содержащем все данные за исключением тестовой выборки): чем больше данных, тем лучше результат. Валидационная выборка теперь не нужна, т.к. мы уже определились с лучшей моделью.

In [43]:
# разделение обучающей выборки dfo_1 на отрицательные и положительные объекты
dfo_1_zeros = dfo_1[dfo_1['Exited']==0]
dfo_1_ones = dfo_1[dfo_1['Exited']==1]
features_1_zeros = dfo_1_zeros.drop(columns=['Exited'])
target_1_zeros = dfo_1_zeros['Exited']
features_1_ones = dfo_1_ones.drop(columns=['Exited'])
target_1_ones = dfo_1_ones['Exited']

In [44]:
# копирование положительных объектов, создание новой обучающей выборки
features_1_sampled = pd.concat([features_1_zeros]+[features_1_ones]*k_best)
target_1_sampled = pd.concat([target_1_zeros]+[target_1_ones]*k_best)

In [45]:
# перемешивание новой обучающей выборки
features_1_sampled, target_1_sampled = shuffle(features_1_sampled, target_1_sampled, random_state = 12345)

In [46]:
# обучение модели
model = RandomForestClassifier(n_estimators=estim_best, max_depth=md_best, random_state=12345)
model.fit(features_1_sampled, target_1_sampled)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=30, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=90,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

#### Проверка модели на тестовой выборке:

In [47]:
# переменные для признаков и целевого признака
features_test = dfo_test.drop(['Exited'], axis=1)
target_test = dfo_test['Exited']

In [48]:
# предсказания для тестовых данных и значения метрик
predicted_test = model.predict(features_test)
prob_test = model.predict_proba(features_test)
prob1_test = pd.Series(prob_test [:, 1])
acc_test = accuracy_score(target_test, predicted_test)
f1_score_test = f1_score(target_test, predicted_test)
roc_auc_score_test = roc_auc_score(target_test, prob1_test)

In [49]:
# подготовка вывода результата на экран
result = best_b
result['accuracy']=acc_test
result['f1_score']=f1_score_test
result['roc_auc_score']=roc_auc_score_test

#### Вывод по результатам финального тестирования

In [50]:
result

,model,n_estimators,depth,method,coefficient,accuracy,f1_score,roc_auc_score
0,random_forest,90,30,upsampling,7.0,0.8565,0.612686,0.863114


F1-мера выбранной модели равна 0.61, т.е. модель удовлетворяет критерию F1-мера >= 0.59.

### 5. Общий вывод.

**Выбранная модель:  
Случайный лес со значением n_estimators = 90, max_depth = 30.  
Учет дисбаланса классов произведен путем увеличения выборки - увеличения количества признаков класса 1 в семь раз.  
F1-мера на тестовой выборке = 0.61.**

Была проделана следующая работа:  
Провели предпоготовку данных, для чего потребовалось заполнить пропуски по признаку "количество объектов недвижимости в собственности".
Подготовили признаки для обучения моделей - произвели кодирование и масштабирование. Использовали прямое (а не порядковое) кодирование, т.к. в числе исследуемых моделей присутствует логистическая регрессия.  
Провели исследование моделей на основе разных алгоритмов: дерево решений, случайный лес и логистическая регрессия. Исследовали модели "дерево решений" и "случайный лес" при различных глубинах дерева. Наилучший результат показала модель "случайный лес". Для этой модели перебрали различные значения параметров n_estimators и random_state.
Исследование баланса классов целевого признака показало, что класс 0 встречается почти в 4 раза чаще, чем класс 1.  
Для учета дисбаланса попробовали использовать 3 разных способа: взвешивание классов, увеличение выборки и уменьшение выборки. Наилучший результат получили при увеличении выборки.  
Перед финальным тестированием обучили модель на выборке, состоящей из тестовых и валидационных данных. 

Также исследовали взаимосвязь метрики AUC-ROC с f1-мерой и долей правильных ответов (accuracy): обнаружена очень сильная корреляция AUC-ROC c accuracy (коэффициент Пирсона = 0.98) и слабая - с f1-мерой (коэффициент Пирсона = 0.6).